In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import cv2
import glob
import matplotlib.pyplot as plt
print(tf.__version__)

In [ ]:
def representative_dataset_gen():
    #folder = "reprDatasetImages"
    folder = "/myReprDataset/"
    image_size = 100
    raw_test_data = []

    files = glob.glob(folder+'/*.jpg')
    for file in files:
        image = Image.open(file)
        image = image.convert("RGB")
        image = image.resize((image_size, image_size))
        #Quantizing the image between -1,1;
        image = (2.0 / 255.0) * np.float32(image) - 1.0
        #image = np.asarray(image).astype(np.float32)
        image = image[np.newaxis,:,:,:]
        raw_test_data.append(image)

    for data in raw_test_data:
        yield [data]

In [ ]:
model = hub.load("https://tfhub.dev/captain-pool/esrgan-tf2/1")
concrete_func = model.signatures[tf.saved_model.DEFAULT_SERVING_SIGNATURE_DEF_KEY]
concrete_func.inputs[0].set_shape([1, 100, 100, 3])
converter = tf.lite.TFLiteConverter.from_concrete_functions([concrete_func])
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8, tf.lite.OpsSet.SELECT_TF_OPS]

#I don't see the following flags making any different effect.
converter.experimental_new_converter = True
converter.experimental_new_quantizer = True

converter.inference_input_type = tf.float32
converter.inference_output_type = tf.float32
converter.allow_custom_ops = False
converter.representative_dataset = representative_dataset_gen
tflite_model = converter.convert()

# Save the TF Lite model.
with tf.io.gfile.GFile('ESRGAN_Quant.tflite', 'wb') as f:
  f.write(tflite_model)

esrgan_model_path = './ESRGAN_Quant.tflite'

In [ ]:
test_img_path = tf.keras.utils.get_file('lr.jpg', 'https://raw.githubusercontent.com/tensorflow/examples/master/lite/examples/super_resolution/android/app/src/main/assets/lr-1.jpg')

In [ ]:
esrgan_model_path = './ESRGAN.tflite'

In [ ]:
lr = cv2.imread('/home/jose/codeWorkspace-2.4.1/tf_2.4.1/superResolution/myReprDataset/000000003501_100x100.jpg')
#lr = cv2.imread('/home/jose/codeWorkspace-2.4.1/tf_2.4.1/superResolution/myReprDataset/cayman.jpg')
lr = cv2.cvtColor(lr, cv2.COLOR_BGR2RGB)
lr = tf.expand_dims(lr, axis=0)
lr = tf.cast(lr, tf.float32)

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=esrgan_model_path)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Run the model
interpreter.set_tensor(input_details[0]['index'], lr)
interpreter.invoke()

# Extract the output and postprocess it
output_data = interpreter.get_tensor(output_details[0]['index'])
sr = tf.squeeze(output_data, axis=0)
sr = tf.clip_by_value(sr, 0, 255)
sr = tf.round(sr)
sr = tf.cast(sr, tf.uint8)

In [ ]:
lr = tf.cast(tf.squeeze(lr, axis=0), tf.uint8)
plt.figure(figsize = (1, 1))
plt.title('LR')
plt.imshow(lr.numpy());

plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)        
plt.title(f'ESRGAN (x4)')
plt.imshow(sr.numpy());

bicubic = tf.image.resize(lr, [1024, 1024], tf.image.ResizeMethod.BICUBIC)
bicubic = tf.cast(bicubic, tf.uint8)
plt.subplot(1, 2, 2)   
plt.title('Bicubic')
plt.imshow(bicubic.numpy());